In [ ]:
# General imports and namespace configuration
import os
import json
import random
from pprint import pprint

%matplotlib inline
import matplotlib.pyplot as plt
import numpy

from nn.activation_functions import *
from nn.cost_funtions import *

from nn import Perceptron

# Perceptron

![Single Perceptron](http://neuralnetworksanddeeplearning.com/images/tikz0.png)

$$ w_{1}x_{1} + w_{2}x_{2} + \dots + w_{i}x_{i} > b $$

$$
\begin{pmatrix}
           w_{1} & w_{2} & \dots & w_{i} 
\end{pmatrix}
\begin{pmatrix}
           x_{1} \\
           x_{2} \\
           \vdots \\
           x_{i}
\end{pmatrix}
> b
$$

$$ \vec{w} \cdot \vec{x} > b $$


$$
\textrm{Evaluacion del perceptron} = 
\begin{cases}
    0 & \textrm{if} & \vec{w} \cdot \vec{x} \leq b \\
    1 & \textrm{if} & \vec{w} \cdot \vec{x} > b
\end{cases}
$$

In [ ]:
x1, x2, x3 = 1, 2, 3
w1, w2, w3 = 5, 87, 8
b = 10

# Perceptron evaluation
result1 = x1 * w1 + x2 * w2 + x3 * w3 - b

# Vectorial evaluation
inputs = numpy.array((x1, x2, x3))
weights = numpy.array((w1, w2, w3))

result2 = numpy.dot(weights, inputs) - b

print(result1, result2)

Heaviside Function
$$
H(x) = 
\begin{cases}
    0 & \textrm{if} & x \leq 0 \\
    1 & \textrm{if} & x > 0
\end{cases}
$$

In [ ]:
x = numpy.linspace(-2, 2)
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)
ax.plot(x, heaviside(x), 'k')
ax.set_xlim(-2, 2)
ax.set_ylim(-1.5, 2.5)
ax.grid()
ax.set_aspect('equal')
ax.set_title("Heaviside fintion")
plt.show()

$$
\textrm{Evaluacion del perceptron} = 
\begin{cases}
    0 & \textrm{if} & \vec{w} \cdot \vec{x} - b \leq 0 \\
    1 & \textrm{if} & \vec{w} \cdot \vec{x} - b > 0
\end{cases}
$$

$$ \textrm{Perceptron} \to H(\vec{w} \cdot \vec{x} - b) $$

## Input space categorizing

for a 2D plane

$$ \vec{w} \cdot \vec{x} - b = w_{1}x_{1} + w_{2}x_{2} - b$$

$$ w_{2}x_{2} = -w_{1}x_{1} + b$$

$$ x_{2} = -\frac{w_{1}}{w_{2}}x_{1} + \frac{b}{w_{2}}$$

$$ y = mx + b$$


In [ ]:
def line(x, m, b):
    return m * x + b 

In [ ]:
weights = numpy.array((-1, 1))
bias = 1

perceptron = Perceptron(weights, bias)

m = -weights[0] / weights[1]
b = bias / weights[1]

samples = [numpy.array((random.uniform(0, 10), random.uniform(0, 10))) for _ in range(500)]

tagged_data = [(sample, perceptron.feedforward(sample)) for sample in samples]
# pprint(tagged_data)

In [ ]:
resolution = 100
x = numpy.linspace(0, 10, resolution)
y = numpy.linspace(0, 10, resolution)
mapped_region = numpy.zeros((resolution, resolution))
for i, x_val in enumerate(x):
    for j, y_val in enumerate(y):
        sample = numpy.array((x_val, y_val))
        mapped_region[i][resolution - 1 - j] = perceptron.feedforward(sample)

In [ ]:
fig = plt.figure(figsize=(10, 5))

subplots = fig.add_subplot(121), fig.add_subplot(122)
for sub in subplots:
    sub.set_aspect('equal')

subplots[0].set_xlim(0, 10)
subplots[0].set_ylim(0, 10)
subplots[1]

for sample, tag in tagged_data:
    if tag:
        color = 'red'
    else:
        color = 'blue'
    subplots[0].plot(sample[0], sample[1], color=color, marker = 'o', markersize=2)


subplots[0].plot(numpy.linspace(0, 10), line(numpy.linspace(0, 10), m, b), 'k--')
subplots[0].set_title('Weights ={1}, {2}'.format(*weights, bias))

subplots[1].imshow(mapped_region, cmap='gray')
subplots[1].set_title('[0, 10]x[0, 10] mapped to [0, 1]')

plt.show()

## Cost functions

In [ ]:
perceptron = Perceptron((0.2, 1), 6)

perceptron_m, perceptron_b = - perceptron.weights[0] / perceptron.weights[1], perceptron.bias / perceptron.weights[1]
print(perceptron_m, perceptron_b)

In [ ]:
# create a dataset
m = -1
b = 10

dataset = []
for _ in range(1000):
    x, y = random.uniform(0, 10), random.uniform(0, 10)
    if y <= m * x + b:
        value = 0.0
    else:
        value = 1.0
    dataset.append((numpy.array((x, y)), value))

In [ ]:
fig = plt.figure(figsize=(17,5))
subplots = fig.add_subplot(131), fig.add_subplot(132), fig.add_subplot(133)

for sample, tag in dataset:
    color = 'red' if tag else 'blue'
    subplots[0].plot(sample[0], sample[1], color=color, marker='o', markersize=2)
subplots[0].plot(numpy.linspace(0, 10), line(numpy.linspace(0, 10), m, b), linestyle='--', label='dataset division')

for sample, _ in dataset:
    tag = perceptron.feedforward(sample)
    color = 'red' if tag else 'blue'
    subplots[1].plot(sample[0], sample[1], color=color, marker='o', markersize=2)
subplots[1].plot(numpy.linspace(0, 10), line(numpy.linspace(0, 10), perceptron_m, perceptron_b), linestyle='--', label='perceptron division')

for sample, data_tag in dataset:
    predicted_tag = perceptron.feedforward(sample)
    color = 'red' if (data_tag and predicted_tag) else 'green' if (data_tag or predicted_tag) else 'blue'
    subplots[2].plot(sample[0], sample[1], color=color, marker='o', markersize=2)
subplots[2].plot(numpy.linspace(0, 10), line(numpy.linspace(0, 10), perceptron_m, perceptron_b), linestyle='--', label='perceptron division')
subplots[2].plot(numpy.linspace(0, 10), line(numpy.linspace(0, 10), m, b), linestyle='--', label='perceptron division')

for sub in subplots:
    sub.set_aspect('equal')
    sub.set_xlim(0, 10)
    sub.set_ylim(0, 10)
    sub.grid()
    sub.legend()

plt.show()

In [ ]:
square_error(dataset, perceptron.feedforward)

In [ ]:
error_history, params_history = perceptron.train(dataset)

perceptron_m, perceptron_b = - perceptron.weights[0] / perceptron.weights[1], perceptron.bias / perceptron.weights[1]
print(perceptron_m, perceptron_b)

In [ ]:
fig = plt.figure(figsize=(11,5))
subplots = fig.add_subplot(121), fig.add_subplot(122)

# Plot the points
for sample, data_tag in dataset:
    predicted_tag = perceptron.feedforward(sample)
    color = 'red' if (data_tag and predicted_tag) else 'green' if (data_tag or predicted_tag) else 'blue'
    subplots[0].plot(sample[0], sample[1], color=color, marker='o', markersize=2)

subplots[0].plot(numpy.linspace(0, 10), line(numpy.linspace(0, 10), perceptron_m, perceptron_b), linestyle='--', label='trained perceptron division')
subplots[0].plot(numpy.linspace(0, 10), line(numpy.linspace(0, 10), m, b), linestyle='--', label='perceptron division')

subplots[1].plot(error_history)

subplots[0].set_xlim(0, 10)
subplots[0].set_ylim(0, 10)


plt.show()

## Non-linear limit

In [ ]:
# create a non linear dataset

non_linear_datasets = [[],[],[]]

for _ in range(1000):
    x, y = random.uniform(-2, 2), random.uniform(-2, 2)
    if x**2 + y**2 <= 1:
        value = 1.0
    else:
        value = 0.0
    non_linear_datasets[0].append((numpy.array((x, y)), value))
    
for _ in range(1000):
    x, y = random.uniform(-numpy.pi, numpy.pi), random.uniform(-2, 2)
    if y > numpy.sin(x):
        value = 1.0
    else:
        value = 0.0
    non_linear_datasets[1].append((numpy.array((x, y)), value))

for _ in range(1000):
    x, y = random.uniform(-2, 2), random.uniform(-1, 3)
    if y > x**2:
        value = 1.0
    else:
        value = 0.0
    non_linear_datasets[2].append((numpy.array((x, y)), value))

In [ ]:
fig = plt.figure(figsize=(17,5))
subplots = fig.add_subplot(131), fig.add_subplot(132), fig.add_subplot(133)

for i, dataset in enumerate(non_linear_datasets):
    subplots[i].grid()
    for sample, tag in dataset:
        color = 'red' if tag else 'blue'
        subplots[i].plot(sample[0], sample[1], color=color, marker='o', markersize=2)

plt.show()